The jacobian of the symmetric matrix eigendecomposition is the vandermonde expression

In [7]:
using LinearAlgebra, GenericLinearAlgebra, ForwardDiff

$$ dA = \prod_{i <j} (\lambda_i - \lambda_j)(d\Lambda)^\wedge(Q'dQ)^\wedge $$

In [70]:
## Grab any nxn symmetric matrix
n = 4
A = randn(n,n);
#A += A'  (not needed, because Hermitian of A will grab the upper triangle anyway)

In [71]:
upper = [i≤j for i=1:n, j=1:n][:]  # upper triangular matrix of 1's , then "vec"ed
Λ, Q =  eigen(Hermitian(A))  # compute the eigendecomposition of A
# now perturb A in each of the n^2 directions and obtain the derivative
# using automatic differentiation technology
dΛ, dQ = (ForwardDiff.jacobian(t -> _eig(Hermitian(t)), A) for _eig=(eigvals, eigvecs))

# dΛ is n x n^2 since we have n eigenvalues perturbed in n^2 directions
# dQ is n^2 x n^2 because we have n matrices of size nxn corresponding to the eigenvectors
# in each of the n^2 directions
# and by the way , each column is the "flattened" or "vec'ed" eigenvector matrix


dΛ, dQ = dΛ[:,upper], dQ[:,upper];
#dΛ # is n by n(n+1)/2 corresponding to n eigenvalues perturbed on the upper triangle
#dQ  # n^2 by n(n+1)/2 corresponding to eigenvectors(nxn matrices flattened) by n(n+1)/2

In [72]:
subupper = [i<j for i=1:n, j=1:n][:]
dQ = (kron(Matrix(I, size(A)),Q')*dQ)[subupper,:]; #compute Q'*dQ for every vec'd dQ

In [73]:
# Experiment vs Theory
det([dΛ;dQ]),1/abs(prod([Λ[i]-Λ[j] for i=1:n, j=1:n if i<j])) # ∂(Λ,Q)/∂A

(0.004692414134163869, 0.004692414134163915)